In [1]:
import pandas as pd
from pathlib import Path

In [2]:
p_root_dir = Path.cwd().parents[1]
p_data = p_root_dir / "5_data"

# Metrics and Catboost predictions for all ions in their target wells
p_predictions = p_data / "all_predictions_curated_11-Dec-2021.csv"

# Dataset info (lab, matrix, polarity, m/z range, ids, etc.)
p_datasets = p_data / r"metadata/Datasets_18Apr2023.csv"

p_out = p_data / "data_for_webapp"

In [3]:
# Load metadata file and select only 3ppm EMBL metadata
datasets = pd.read_csv(p_datasets, index_col=0)
metadata = datasets[datasets.EMBL]

# Drop some columns that are not needed and make sure there is only one row per dataset
cols_to_drop = ['Dataset name', 'm/z range', 'Pixel size', 'Original Dataset ID (3ppm)', 'Original link',
       'Dataset link', 'EMBL', 'Interlab', 'All']
metadata.drop(columns=cols_to_drop, inplace=True)
metadata.drop_duplicates(inplace=True)

<ipython-input-3-8ba655c402c6>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  metadata.drop(columns=cols_to_drop, inplace=True)
<ipython-input-3-8ba655c402c6>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  metadata.drop_duplicates(inplace=True)


In [4]:
# Load predictions, format neutral loss column
predictions = pd.read_csv(p_predictions, index_col=0)
predictions.neutral_loss.fillna('', inplace=True)
predictions.score.fillna("", inplace=True)

# Merge metadata_df with concatenated_df on dataset_id
final_df = pd.merge(metadata, predictions, right_on="dataset_id", left_on="Dataset ID").drop(columns=['Dataset ID'])

# Save input for EMBL data tab
final_df.to_csv(p_out / "EMBL_data_22May2023.csv", index=False)